# 07. Strategic Insights & Clustering

In [1]:
import sys
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project Root: {project_root}")


Project Root: c:\Users\Kaustab das\Desktop\Aadhaar Pulse AI


In [2]:
# Load Processed Data
processed_path = os.path.join(project_root, 'data', 'processed', 'merged_master_table.csv')
if os.path.exists(processed_path):
    df = pd.read_csv(processed_path)
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    print(f"Data Loaded: {df.shape}")
else:
    print("Error: Processed data not found. Please run run_pipeline.py first.")


Data Loaded: (2947681, 15)


## 7.1 Aadhaar Update Stress Index (AUSI)

In [3]:
from src.metrics.stress_index import calculate_ausi

df = calculate_ausi(df)
top_stress = df.sort_values('ausi_score', ascending=False)[['state', 'district', 'date', 'ausi_score']].head(10)
print("Districts with Peak AUSI:")
display(top_stress)


Districts with Peak AUSI:


,state,district,date,ausi_score
1250440,Haryana,Mahendragarh,2025-03-01,100.0
1351687,Kerala,Pathanamthitta,2025-07-01,100.0
1351828,Andhra Pradesh,Krishna,2025-07-01,100.0
1351829,Punjab,Fatehgarh Sahib,2025-07-01,100.0
1351830,Orissa,Bhadrak,2025-07-01,100.0
1351831,Telangana,Jogulamba Gadwal,2025-07-01,100.0
1351832,Telangana,Jayashankar Bhupalpally,2025-07-01,100.0
1351833,Gujarat,Junagadh,2025-07-01,100.0
1351834,Tamil Nadu,Viluppuram,2025-07-01,100.0
1351835,Uttar Pradesh,Ayodhya,2025-07-01,100.0


## 7.2 Migration Analysis

In [4]:
from src.analytics.migration_analysis import calculate_migration_score

df = calculate_migration_score(df)
# Top Migration Flux districts
top_mig = df.groupby(['state', 'district'])['migration_flux'].mean().sort_values(ascending=False).head(10)
print("Top Migration Hubs:")
print(top_mig)


Top Migration Hubs:
state         district        
Delhi         North East Delhi    159.677580
              North Delhi         141.823997
Chhattisgarh  Mahasamund          127.288908
Delhi         North West Delhi     97.618934
Chhattisgarh  Narayanpur           93.675766
              Kawardha             90.398707
Delhi         East Delhi           84.709002
              West Delhi           83.400044
Chhattisgarh  Rajnandgaon          77.806785
              Mungeli              66.998386
Name: migration_flux, dtype: float64


## 7.3 Infrastructure Recommendations (Clustering)

In [8]:
from src.models.clustering import cluster_districts

recommendations = cluster_districts(df)

print("Infrastructure Recommendations Count:")
print(recommendations['recommendation'].value_counts())

print("\nTop 10 High Priority New Centers Required:")
new_centers = recommendations[recommendations['recommendation'] == 'New Center Required']
display(new_centers.sort_values('impact_score', ascending=False).head(10))

Infrastructure Recommendations Count:
recommendation
Monitor                                693
Mobile Van Required                    334
New Center Required                     91
Mobile Van Required (Migration Hub)     14
Name: count, dtype: int64

Top 10 High Priority New Centers Required:


,state,district,ausi_score,total_updates,total_enrolment,migration_flux,cluster,recommendation,impact_score
591,Maharashtra,Yavatmal,44.246854,580097.0,9125.0,34.606030,2,New Center Required,100.000000
229,Delhi,North West Delhi,23.713820,751291.0,21444.0,97.618934,2,New Center Required,69.410715
571,Maharashtra,Nanded,26.027085,638581.0,14201.0,26.712764,2,New Center Required,64.752783
550,Maharashtra,Chandrapur,42.526508,357625.0,5412.0,20.168143,2,New Center Required,59.252277
539,Maharashtra,Ahmadnagar,23.246964,651949.0,15404.0,15.480619,2,New Center Required,59.046854
543,Maharashtra,Amravati,29.557224,488045.0,8925.0,13.030224,2,New Center Required,56.200558
565,Maharashtra,Latur,34.871495,408822.0,9147.0,26.764168,2,New Center Required,55.542077
545,Maharashtra,Beed,28.548065,475086.0,12639.0,28.874047,2,New Center Required,52.840395
586,Maharashtra,Solapur,20.273539,659718.0,15103.0,21.090187,2,New Center Required,52.108048
574,Maharashtra,Nashik,14.215247,922715.0,28084.0,16.750224,2,New Center Required,51.102093
